# Triggers
A trigger: a database program, consisting of procedural and declarative instructons, saved in the catalogue and activated by the DBMS if a certain operation on the database is executed and if a certain condition is satisfied.
- Comparable to SP but can't be called explicitly
- Automatically called by the DBMS on a certain action
> In this course we'll focus on DML mutation triggers.

## Activation
- **`before`** – before the IUD is processed **(not supported by MS SQL Server)**
- **`instead of`**  –  instead of IUD command
- **`after`** – after the IUID is processed (but before COMMIT)
    > `After`  is the default

## Use cases 
- Validation of data and complex constraints
    - An employee can't be assigned to > 10 projects
    - An employee can only be assigned to a project that is assigned to his department
- Automatic generation of values
    - If an employee is assigned to a project the default value for the monthly bonus is set according to the project priority and his job category
- Support for alerts
    - Send automatic e-mail if an employee is removed from a project
- Auditing
    - Keep track of who did what on a certain table
    - Replication and controlled update of redundant data
    - Db xtreme: If an ordersdetail record changes, update the orderamount in the orders table
    - Automatic update of datawarehouse tables for reporting (see "Datawarehousing")

## Advantages
- Store functionality in the DB and execute **consistently** with each change of data in the DB
- no redundant code
    - functionality is localised in a single spot, not scattered over different applications (desktop, web, mobile), written by different authors
- written & tested 'once' by an experienced DBA
- security
    - triggers are in the DB so all security rules apply
## Consequences
- more processing power
- fits into client-server model
- 1 call to db-serve: al lot can happen without further communication

## Drawbacks
- complexity
    - DB design, implementation are more complex by shifting functionality from application to DB
    - Very difficult to debug
- Hidden functionality
    - The user can be confronted with unexpected side effects from the trigger, possibly unwanted
    - Triggers can cascade, which is not always easy to predict when designing the trigger
- Performance
    - At each database change the triggers have to be reevaluated
- Portability
    - Restricted to the chosen database dialect (ex. Transact-SQL from MS)

## Inner workings
- 2 temporary tables
- `deleted` table
    - contains copies of updated and deleted rows
    - During update or delete rows are moved from the triggering table to the deleted table
    - Those two table have no rows in common

- `inserted` table
    - contains copies of updated or inserted rows
    - During update or insert each affected row is copied from the triggering table to the inserted table
    - All rows from the inserted table are also in the triggering table


## Syntax
```sql
CREATE TRIGGER [Name of the trigger]
ON table
FOR [INSERT, UPDATE, DELETE]
AS [Statement here]
```

## Example





### Insert afrer-trigger

In [ ]:
USE TENNIS;
CREATE TRIGGER tr_insert_player ON PLAYERS FOR INSERT
AS
 INSERT INTO mutation (username, mut_timestamp, mut_pnr, mut_type, mut_pnr_new)
 SELECT user, getdate(), null, 'i', playerno FROM inserted


- triggering instruction is an insert statement 
- `inserted` – logical table with columns equal to columns of triggering table, containing a copy of inserted rows
> Remark: when triggering by INSERT-SELECT statement more than one record can be added at once. The trigger code is executed only once, but will insert a mutation record for each inserted record



### Delete after-trigger
Before we continue with this trigger let's first create a Stored Procedure to re-use some logic.

In [ ]:
CREATE PROCEDURE usp_mutation_insert
  (@MPNR SMALLINT,
   @MTYPE CHAR(1),
   @MPNR_NEW SMALLINT) 
AS
  INSERT INTO mutation 
  (username, mut_timestamp, mut_pnr,mut_type, mut_pnr_new) 
  VALUES (user, getdate(), @MPNR, @MTYPE, @MPNR_NEW);


In [ ]:
CREATE TRIGGER  tr_delete_player
ON players FOR DELETE
AS
    DECLARE @old_pnr smallint
    DECLARE del_cursor CURSOR FOR SELECT playerno FROM deleted
    OPEN del_cursor
    FETCH NEXT FROM del_cursor INTO @old_pnr
    WHILE @@FETCH_STATUS = 0
    BEGIN
        EXEC usp_mutation_insert @old_pnr,'D',null
        FETCH NEXT FROM del_cursor INTO @old_pnr
    END
    CLOSE del_cursor
    DEALLOCATE del_cursor


Activation of the Trigger

In [ ]:
DELETE FROM players WHERE playerno > 115;

### Update after-trigger

In [ ]:
CREATE TRIGGER  tr_update_player ON players FOR update
AS
DECLARE 
 @old_pnr smallint
,@new_pnr smallint

DECLARE before_cursor CURSOR FOR SELECT playerno FROM DELETED  ORDER BY playerno
DECLARE after_cursor  CURSOR FOR SELECT playerno FROM INSERTED ORDER BY playerno

OPEN before_cursor
OPEN after_cursor

FETCH NEXT FROM before_cursor INTO @old_pnr
FETCH NEXT FROM after_cursor  INTO @new_pnr

WHILE @@FETCH_STATUS = 0
BEGIN
    EXEC usp_mutation_insert @old_pnr,'U',@new_pnr
    FETCH NEXT FROM before_cursor INTO @old_pnr
    FETCH NEXT FROM after_cursor  INTO @new_pnr
END

DEALLOCATE before_cursor
DEALLOCATE after_cursor


Activation of the Trigger

In [ ]:
update players set joined = joined + 20;

-- OR:

 update players set playerno = playerno + 100;


## Conditional execution of triggers
Execute only if a specific column is mentioned in update or insert


In [ ]:
ALTER TRIGGER  update_player ON players FOR update
AS
DECLARE @old_pnr smallint, @new_pnr smallint

DECLARE before_cursor CURSOR FOR SELECT playerno FROM deleted  ORDER BY playerno
OPEN before_cursor
IF update(playerno)
BEGIN
  DECLARE after_cursor CURSOR FOR SELECT playerno FROM inserted ORDER BY playerno
  OPEN after_cursor
END

FETCH NEXT FROM before_cursor INTO @old_pnr
IF update(playerno)
  FETCH NEXT FROM after_cursor  INTO @new_pnr
ELSE
  SET @new_pnr = @old_pnr

WHILE @@FETCH_STATUS = 0
BEGIN
  EXEC usp_mutation_insert @old_pnr,'U',@new_pnr
  FETCH NEXT FROM before_cursor INTO @old_pnr
  IF update(playerno)
    FETCH NEXT FROM after_cursor  INTO @new_pnr
  ELSE
    SET @new_pnr = @old_pnr
END

DEALLOCATE before_cursor
IF update(playerno)
  DEALLOCATE after_cursor


# Triggers and transactions
a trigger is part of the same transaction as the triggering instruction
inside the trigger this transaction can be ROLLBACKed
ex. a player who is team coach can never be deleted (suppose there are no foreign key constraints)
 although a trigger in SQL Server occurs after the triggering instruction, that instruction can still be undone in the trigger


In [ ]:
CREATE TRIGGER delplayer ON PLAYERS 
FOR delete 
AS 
IF (SELECT COUNT(*) 
    FROM deleted JOIN teams ON teams.playerno = deleted.playerno) > 0 
BEGIN 
  ROLLBACK TRANSACTION
  RAISERROR ('The player is team coach.', 14,1) 
END
